In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanicx/titanic.csv


In [132]:
df = pd.read_csv("../input/titanicx/titanic.csv")

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [1]:
from helpers import *

/kaggle/input/titanicx/titanic.csv


In [5]:
grab_col_names(df)

Observations: 891
Variables: 12
cat_cols: 6
num_cols: 3
cat_but_car: 3
num_but_cat: 4


(['Sex', 'Embarked', 'Survived', 'Pclass', 'SibSp', 'Parch'],
 ['PassengerId', 'Age', 'Fare'],
 ['Name', 'Ticket', 'Cabin'])

In [133]:
def titanic_data_prep(data):
    print("############## LABEL ENCODING ##############")
    binary_col = [col for col in df.columns if df[col].dtypes not in [float, int] and df[col].nunique() == 2]
    print(binary_col, label_encoder(data, binary_col), end="\n\n\n")
    
    print("############## OUTLIERS ##############")
    num_cols = [col for col in data.columns if data[col].dtypes != "O"]
    for col_name in num_cols:
        outlier_threshold(data, col_name, w1=0.05, w2=0.95)
    print("############## are there any outliers? ##############")        
    for col_name in num_cols:
            if check_outlier(data, col_name, w1=0.05, w2=0.95):
                print(col_name)
                print(col_name, grab_outliers(data, col_name, index=False, w1=0.05, w2=0.95))
                remove_outlier(data, col_name, w1=0.05, w2=0.95)
                print(col_name, "outliers_removed")
            else:
                print(col_name, "no outliers")
    print("\n\n\n")            
    print("############## MISSING VALUES ##############")
    print(data.isnull().sum())
    print(missing_values_table(data, na_name=True),  end="\n\n\n")
        
        
    print("############## ONE HOT ENCODER ##############")
    categorical_cols = [col for col in df.columns if df[col].dtypes == "O" and 10 >= df[col].nunique() > 2]
    print(categorical_cols, one_hot_encoder(data, categorical_cols, drop_first=True),  end="\n\n\n")
    
    print("############## FINDING RARE VALUES ##############")
    cat_cols = [col for col in data.columns if data[col].dtype not in ["int", "float"]]
    rare_analyser(data, "Survived", cat_cols)
    print("\n\n\n")        
    print("############## RARE ENCODING ##############")        
    new_df = rare_encoder(data, 0.01)
    new_df.shape
    rare_analyser(new_df, "Survived", cat_cols)

In [134]:
new_df = titanic_data_prep(df)

############## LABEL ENCODING ##############
['Sex']      PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name  Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    1  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...    0  38.0      1   
2                               Heikkinen, Miss. Laina    0  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)    0  35.0      1   
4                             Allen, Mr. William Henry    1  35.0      0   
..                                

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [141]:
import pickle
outfile=open("titanic_prep.pkl","wb")


In [142]:
pickle.dump(new_df,outfile)